# **Задание 1**

In [23]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00


In [25]:
# предобработка текста
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML
import matplotlib.pyplot as plt
%matplotlib inline
from datasets import load_dataset
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv('lenta_40k.csv.zip')

def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

vocab = Counter()

for text in data.text:
    vocab.update(preprocess(text))

filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

word2id = {'PAD':0, 'UNK':1}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

MAX_LEN = max(len(x) for x in X)

MEAN_LEN = np.median([len(x) for x in X])

MAX_LEN = int(MEAN_LEN + 30)

X = keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

y = keras.utils.to_categorical([label2id[label] for label in data.topic.values])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

In [3]:
# callback-параметры

checkpoint = keras.callbacks.ModelCheckpoint('model.weights.h5',
                                                monitor='recall@precision',
                                                verbose=1,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='max',
                                                save_freq='epoch'
                                               )

# для остановки
early_stop = keras.callbacks.EarlyStopping(monitor='val_rec@prec',
                                              min_delta=0.01,
                                              patience=5,
                                           # взяла побольше; пусть терпит,
                                           # потому что господь с ним еще не закончил
                                              verbose=1,
                                              mode='max',
                                              )

In [5]:
# модель 1
inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs)

# 2 реккурентных слоя; во втором случае передается только последнее состояние
# RNN
rnn_LSTM = keras.layers.LSTM(128, return_sequences=True)(embeddings)
rnn_GPU = keras.layers.GRU(128, return_sequences=True)(rnn_LSTM)

# пулинг
ap_layer = keras.layers.GlobalAveragePooling1D()(rnn_GPU)
mp_layer = keras.layers.GlobalMaxPool1D()(rnn_GPU)
concat = keras.layers.concatenate([rnn_GPU[:,-1,:], ap_layer, mp_layer])

outputs = keras.layers.Dense(len(label2id), activation='softmax')(concat)

model1 = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model1.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 200)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 200, 30)        │        722,790 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 200, 128)       │         81,408 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru (GRU)                 │ (None, 200, 128)       │         99,072 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item (GetItem)        │ (None, 128)            │              0 │ gru[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 128)            │              0 │ gru[0][0]              │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d      │ (None, 128)            │              0 │ gru[0][0]              │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 384)            │              0 │ get_item[0][0],        │
│                           │                        │                │ global_average_poolin… │
│                           │                        │                │ global_max_pooling1d[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 19)             │          7,315 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 910,585 (3.47 MB)

 Trainable params: 910,585 (3.47 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# обучение модели 1
model1.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=200,
          epochs=10,
          callbacks=[checkpoint, early_stop])

Epoch 1/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 132s 615ms/step - loss: 2.3422 - rec@prec: 0.0072 - val_loss: 1.4641 - val_rec@prec: 0.2723
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with recall@precision available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


211/211 ━━━━━━━━━━━━━━━━━━━━ 123s 582ms/step - loss: 1.2390 - rec@prec: 0.4088 - val_loss: 1.1577 - val_rec@prec: 0.4883
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 120s 571ms/step - loss: 0.7936 - rec@prec: 0.7043 - val_loss: 1.0517 - val_rec@prec: 0.5487
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 123s 583ms/step - loss: 0.5593 - rec@prec: 0.8538 - val_loss: 1.0750 - val_rec@prec: 0.5798
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 121s 574ms/step - loss: 0.4191 - rec@prec: 0.9187 - val_loss: 1.1507 - val_rec@prec: 0.5654
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 120s 570ms/step - loss: 0.3167 - rec@prec: 0.9558 - val_loss: 1.2930 - val_rec@prec: 0.5410
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 120s 569ms/step - loss: 0.2437 - rec@prec: 0.9728 - val_loss: 1.4728 - val_rec@prec: 0.5329
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 120s 570ms/step - loss: 0.1921 - rec@prec: 0.9830 - val_loss: 1.6035 - val_rec@prec: 0.5162
Epoch 9/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 119s 566ms/step - loss: 0.1444 - rec@prec: 0.98

In [17]:
# результаты модели 1

pred_model1 = model1.predict(X_valid)
print(classification_report(y_valid.argmax(1),
                             pred_model1.argmax(1),
                             zero_division=0))

70/70 ━━━━━━━━━━━━━━━━━━━━ 23s 331ms/step
              precision    recall  f1-score   support

           0       0.37      0.37      0.37        84
           1       0.82      0.68      0.74       239
           2       0.19      0.27      0.23        22
           3       0.70      0.71      0.71       410
           4       0.43      0.37      0.40       132
           5       0.00      0.00      0.00         2
           6       0.66      0.66      0.66       160
           7       0.40      0.29      0.33        21
           8       0.56      0.62      0.59        66
          10       0.32      0.30      0.31        60
          11       0.00      0.00      0.00         1
          13       0.65      0.77      0.70       481
          14       0.83      0.67      0.74       159
          15       0.00      0.00      0.00         4
          16       0.93      0.91      0.92       195
          17       0.59      0.60      0.60       159
          18       0.47      0.35      

In [12]:
# модель 2
inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs)

# 2 реккурентных слоя; во втором случае передается только последнее состояние
# RNN
rnn_LSTM_bi = keras.layers.Bidirectional(keras.layers.LSTM(128,
                                                        return_sequences=True))(embeddings)
rnn_GPU_bi = keras.layers.Bidirectional(keras.layers.GRU(128,
                                                         return_sequences=True))(rnn_LSTM_bi)

# пулинг
ap_layer = keras.layers.GlobalAveragePooling1D()(rnn_GPU_bi)
mp_layer = keras.layers.GlobalMaxPool1D()(rnn_GPU_bi)
concat = keras.layers.concatenate([rnn_GPU_bi[:,-1,:], ap_layer, mp_layer])

outputs = keras.layers.Dense(len(label2id), activation='softmax')(concat)

model2 = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[keras.metrics.RecallAtPrecision(0.8, name='rec@prec')])

model2.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 200)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 200, 30)        │        722,790 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 200, 256)       │        162,816 │ embedding_1[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 200, 256)       │        296,448 │ bidirectional[0][0]    │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_1 (GetItem)      │ (None, 256)            │              0 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 256)            │              0 │ bidirectional_1[0][0]  │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_1    │ (None, 256)            │              0 │ bidirectional_1[0][0]  │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 768)            │              0 │ get_item_1[0][0],      │
│ (Concatenate)             │                        │                │ global_average_poolin… │
│                           │                        │                │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 19)             │         14,611 │ concatenate_1[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,196,665 (4.56 MB)

 Trainable params: 1,196,665 (4.56 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# обучение модели 2
model2.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=200,
          epochs=10,
          callbacks=[checkpoint, early_stop])

Epoch 1/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - loss: 2.1930 - rec@prec: 0.0338 - val_loss: 1.2964 - val_rec@prec: 0.3490
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with recall@precision available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


211/211 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - loss: 1.0435 - rec@prec: 0.5415 - val_loss: 0.9854 - val_rec@prec: 0.6082
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 252s 1s/step - loss: 0.6395 - rec@prec: 0.8153 - val_loss: 0.9883 - val_rec@prec: 0.5902
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - loss: 0.4307 - rec@prec: 0.9110 - val_loss: 1.0234 - val_rec@prec: 0.6294
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - loss: 0.2733 - rec@prec: 0.9668 - val_loss: 1.1560 - val_rec@prec: 0.5974
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 249s 1s/step - loss: 0.1732 - rec@prec: 0.9866 - val_loss: 1.3268 - val_rec@prec: 0.5794
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - loss: 0.1149 - rec@prec: 0.9935 - val_loss: 1.5513 - val_rec@prec: 0.5816
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - loss: 0.0889 - rec@prec: 0.9965 - val_loss: 1.6780 - val_rec@prec: 0.5613
Epoch 9/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 253s 1s/step - loss: 0.0659 - rec@prec: 0.9982 - val_loss: 1.8259 - 

In [14]:
# результаты модели 2

pred_model2 = model2.predict(X_valid)
print(classification_report(y_valid.argmax(1),
                             pred_model2.argmax(1),
                             zero_division=0))

70/70 ━━━━━━━━━━━━━━━━━━━━ 41s 579ms/step
              precision    recall  f1-score   support

           0       0.38      0.32      0.35        84
           1       0.81      0.78      0.79       239
           2       0.40      0.27      0.32        22
           3       0.70      0.71      0.70       410
           4       0.48      0.58      0.53       132
           5       0.00      0.00      0.00         2
           6       0.81      0.69      0.75       160
           7       0.20      0.29      0.24        21
           8       0.68      0.64      0.66        66
          10       0.50      0.28      0.36        60
          11       0.00      0.00      0.00         1
          13       0.69      0.74      0.71       481
          14       0.74      0.78      0.76       159
          15       1.00      0.25      0.40         4
          16       0.90      0.93      0.91       195
          17       0.72      0.65      0.68       159
          18       0.47      0.65      

# **Задание 2**

## Подготовка данных

In [27]:
dataset = load_dataset("tner/wikiann", 'ru')

The repository for tner/wikiann contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/tner/wikiann.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


test.jsonl:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/5.26M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [44]:
# подготовка данных для обучения
vocab = Counter()

for sent in dataset['train']['tokens']:
    vocab.update([x.lower() for x in sent])

word2id = {'PAD':0, 'UNK':1}

for word in vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

X = []

for sent in dataset['train']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

X_test = []

for sent in dataset['test']['tokens']:
    tokens = [w.lower() for w in sent]
    ids = [word2id.get(token, 1) for token in tokens]
    X_test.append(ids)


MAX_LEN = max(len(x) for x in X)
# паддинг
X = keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN, padding='post')
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=MAX_LEN, padding='post')

label2id = {
    "B-LOC": 0,
    "B-ORG": 1,
    "B-PER": 2,
    "I-LOC": 3,
    "I-ORG": 4,
    "I-PER": 5,
    "O": 6,
    "PAD": 7
}
id2labels = {v:k for k,v in label2id.items()}

y = keras.preprocessing.sequence.pad_sequences(dataset['train']['tags'], value=7,
                                                  maxlen=MAX_LEN,  padding='post')
y_test = keras.preprocessing.sequence.pad_sequences(dataset['test']['tags'], value=7,
                                                       maxlen=MAX_LEN,  padding='post')

X.shape, X_test.shape
y.shape, y_test.shape

((20000, 54), (10000, 54))

In [52]:
# secret tool we'll use later

import re

def tokenize(text, word2id):
    # токенизирует и переводит в индексы
    tokens = re.findall('\w+|[^\w\s]+', text)
    ids = [word2id.get(token.lower(), 1) for token in tokens]
    return tokens, ids

def pred2tags(pred, id2label, length):
    # декодирует индексы в части речи
    # length нужно чтобы откидывать паддинги или некорректные предсказания
    pred = pred.argmax(2)[0, :length]
    labels = [id2label[l] for l in pred]
    return labels

def label_seq(text, word2id, id2label, max_len, model):
    tokens, ids = tokenize(text, word2id)
    pred = model.predict(keras.preprocessing.sequence.pad_sequences([ids],
                                                                       maxlen=max_len,
                                                                       padding='post'))
    labels = pred2tags(pred, id2label, len(ids))

    return list(zip(tokens, labels))

## Модель 1

In [45]:
# модель wk1

inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

lstm_wk1 = keras.layers.LSTM(128, return_sequences=True)(embeddings)
gru_wk1 = keras.layers.GRU(128, return_sequences=True)(lstm_wk1)

outputs = keras.layers.Dense(len(label2id), activation='softmax')(gru_wk1)

model_wk1 = keras.Model(inputs=inputs, outputs=outputs)
model_wk1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model_wk1.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 54)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_3 (Embedding)              │ (None, 54, 100)             │       3,601,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 54, 128)             │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 54, 128)             │          99,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 54, 8)               │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,819,052 (14.57 MB)

 Trainable params: 3,819,052 (14.57 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model_wk1.fit(X, y,
              validation_data=(X_test, y_test),
              batch_size=1028,
              epochs=10,
              callbacks=[checkpoint, early_stop])

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 207ms/step - accuracy: 0.7217 - loss: 1.4058 - val_accuracy: 0.8680 - val_loss: 0.4104
Epoch 2/10
 1/20 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step - accuracy: 0.8694 - loss: 0.4133

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with recall@precision available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_rec@prec` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 207ms/step - accuracy: 0.8694 - loss: 0.3478 - val_accuracy: 0.8689 - val_loss: 0.2936
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 240ms/step - accuracy: 0.8707 - loss: 0.2855 - val_accuracy: 0.8735 - val_loss: 0.2717
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 209ms/step - accuracy: 0.8974 - loss: 0.2616 - val_accuracy: 0.9258 - val_loss: 0.2502
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 213ms/step - accuracy: 0.9282 - loss: 0.2387 - val_accuracy: 0.9336 - val_loss: 0.2327
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 253ms/step - accuracy: 0.9351 - loss: 0.2199 - val_accuracy: 0.9366 - val_loss: 0.2204
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 210ms/step - accuracy: 0.9384 - loss: 0.2035 - val_accuracy: 0.9384 - val_loss: 0.2105
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step - accuracy: 0.9408 - loss: 0.1911 - val_accuracy: 0.9409 - val_loss: 0.2020
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 219ms/step - accuracy: 0.9442 - loss: 0.1783 - val_accuracy: 0.9435 - val_

In [62]:
# результаты модели wk1
pred_model_wk1 = model_wk1.predict(X_test).argmax(2)

print(classification_report(y_test.reshape(-1), pred_model_wk1.reshape(-1),
                            labels=list(id2labels.keys()),
                            target_names=list(id2labels.values()),
                            zero_division=0))

313/313 ━━━━━━━━━━━━━━━━━━━━ 36s 114ms/step
              precision    recall  f1-score   support

       B-LOC       0.25      0.00      0.00      4560
       B-ORG       0.00      0.00      0.00      4074
       B-PER       0.00      0.00      0.00      3542
       I-LOC       0.00      0.00      0.00      3060
       I-ORG       0.33      0.35      0.34      8008
       I-PER       0.84      0.04      0.08      7544
           O       0.65      0.96      0.77     40480
         PAD       0.99      1.00      1.00    468732

    accuracy                           0.95    540000
   macro avg       0.38      0.29      0.27    540000
weighted avg       0.93      0.95      0.93    540000



### Предсказания модели 1

In [54]:
label_seq('Лада припарковалась возле торгового центра.', word2id, id2labels,
          MAX_LEN, model_wk1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step


[('Лада', 'O'),
 ('припарковалась', 'O'),
 ('возле', 'O'),
 ('торгового', 'O'),
 ('центра', 'O'),
 ('.', 'O')]

In [67]:
label_seq('Том Пушкина был поставлен на самое видное место', word2id, id2labels,
          MAX_LEN, model_wk1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


[('Том', 'O'),
 ('Пушкина', 'PAD'),
 ('был', 'PAD'),
 ('поставлен', 'O'),
 ('на', 'O'),
 ('самое', 'O'),
 ('видное', 'O'),
 ('место', 'O')]

In [56]:
label_seq('Вышка должна была столкнуться с рядом неприятностей.', word2id, id2labels,
          MAX_LEN, model_wk1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step


[('Вышка', 'O'),
 ('должна', 'O'),
 ('была', 'O'),
 ('столкнуться', 'O'),
 ('с', 'O'),
 ('рядом', 'O'),
 ('неприятностей', 'O'),
 ('.', 'O')]

Либо я что-то сделала не так, либо оно, эээ, не очень работает (точнее, вообще не работает).

## Модель 2

In [60]:
# модель wk2

inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

lstm_wk1 = keras.layers.LSTM(128, return_sequences=True)(embeddings)
gru_wk1 = keras.layers.GRU(128, return_sequences=True)(lstm_wk1)


lstm_wk2_1 = keras.layers.LSTM(100, return_sequences=True)(embeddings)
lstm_wk2_2 = keras.layers.LSTM(100, return_sequences=True)(lstm_wk2_1)
lstm_wk2_12 = keras.layers.concatenate([lstm_wk2_2, embeddings])
lstm_wk2_3 = keras.layers.LSTM(100, return_sequences=True)(lstm_wk2_12)
lstm_wk2_123 = keras.layers.concatenate([lstm_wk2_3, embeddings])

outputs = keras.layers.Dense(len(label2id), activation='softmax')(gru_wk1)

model_wk2 = keras.Model(inputs=inputs, outputs=outputs)
model_wk2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model_wk2.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 54)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_7 (Embedding)              │ (None, 54, 100)             │       3,601,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ (None, 54, 128)             │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_7 (GRU)                          │ (None, 54, 128)             │          99,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 54, 8)               │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,819,052 (14.57 MB)

 Trainable params: 3,819,052 (14.57 MB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model_wk2.fit(X, y,
              validation_data=(X_test, y_test),
              batch_size=1028,
              epochs=10,
              callbacks=[checkpoint, early_stop])

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 210ms/step - accuracy: 0.7213 - loss: 1.3410 - val_accuracy: 0.8680 - val_loss: 0.3972
Epoch 2/10
 1/20 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - accuracy: 0.8682 - loss: 0.3879

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with recall@precision available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_rec@prec` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 206ms/step - accuracy: 0.8697 - loss: 0.3412 - val_accuracy: 0.8689 - val_loss: 0.2925
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 236ms/step - accuracy: 0.8702 - loss: 0.2852 - val_accuracy: 0.8995 - val_loss: 0.2669
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 206ms/step - accuracy: 0.9104 - loss: 0.2576 - val_accuracy: 0.9318 - val_loss: 0.2439
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 216ms/step - accuracy: 0.9316 - loss: 0.2352 - val_accuracy: 0.9348 - val_loss: 0.2278
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 231ms/step - accuracy: 0.9365 - loss: 0.2164 - val_accuracy: 0.9360 - val_loss: 0.2163
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 207ms/step - accuracy: 0.9386 - loss: 0.2027 - val_accuracy: 0.9383 - val_loss: 0.2079
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 5s 232ms/step - accuracy: 0.9416 - loss: 0.1895 - val_accuracy: 0.9408 - val_loss: 0.2007
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 207ms/step - accuracy: 0.9443 - loss: 0.1790 - val_accuracy: 0.9434 - val_

In [63]:
# результаты модели wk1
pred_model_wk2 = model_wk2.predict(X_test).argmax(2)

print(classification_report(y_test.reshape(-1), pred_model_wk2.reshape(-1),
                            labels=list(id2labels.keys()),
                            target_names=list(id2labels.values()),
                            zero_division=0))

313/313 ━━━━━━━━━━━━━━━━━━━━ 26s 83ms/step
              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00      4560
       B-ORG       0.00      0.00      0.00      4074
       B-PER       0.00      0.00      0.00      3542
       I-LOC       0.00      0.00      0.00      3060
       I-ORG       0.34      0.38      0.36      8008
       I-PER       0.59      0.01      0.01      7544
           O       0.63      0.97      0.77     40480
         PAD       1.00      1.00      1.00    468732

    accuracy                           0.95    540000
   macro avg       0.32      0.29      0.27    540000
weighted avg       0.93      0.95      0.93    540000



### Предсказания модели 2

In [65]:
label_seq('Лада припарковалась возле торгового центра.', word2id, id2labels,
          MAX_LEN, model_wk2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


[('Лада', 'O'),
 ('припарковалась', 'O'),
 ('возле', 'O'),
 ('торгового', 'O'),
 ('центра', 'O'),
 ('.', 'O')]

In [66]:
label_seq('Том Пушкина был поставлен на самое видное место', word2id, id2labels,
          MAX_LEN, model_wk2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step


[('Том', 'O'),
 ('Пушкина', 'PAD'),
 ('был', 'PAD'),
 ('поставлен', 'PAD'),
 ('на', 'O'),
 ('самое', 'O'),
 ('видное', 'PAD'),
 ('место', 'O')]

In [68]:
label_seq('Вышка должна была столкнуться с рядом неприятностей.', word2id, id2labels,
          MAX_LEN, model_wk1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


[('Вышка', 'O'),
 ('должна', 'O'),
 ('была', 'O'),
 ('столкнуться', 'O'),
 ('с', 'O'),
 ('рядом', 'O'),
 ('неприятностей', 'O'),
 ('.', 'O')]

Ну ээээ... Тоже так себе определяет

## Модель 3

In [74]:
# модель wk3

inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=100)(inputs)

lstm_wk3 = keras.layers.LSTM(128, return_sequences=True)(embeddings)
gru_wk3 = keras.layers.GRU(128, return_sequences=True)(lstm_wk3)

conv1_wk3 = keras.layers.Conv1D(kernel_size=2, filters=128, strides=1, activation='relu',
                                padding='same')(lstm_wk3)
conv2_wk3 = keras.layers.Conv1D(kernel_size=2, filters=128, strides=1, activation='relu',
                                padding='same')(conv1_wk3)


outputs = keras.layers.Dense(len(label2id), activation='softmax')(conv2_wk3)

model_wk3 = keras.Model(inputs=inputs, outputs=outputs)
model_wk3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model_wk3.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 54)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_11 (Embedding)             │ (None, 54, 100)             │       3,601,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_22 (LSTM)                       │ (None, 54, 128)             │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_6 (Conv1D)                    │ (None, 54, 128)             │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_7 (Conv1D)                    │ (None, 54, 128)             │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 54, 8)               │           1,032 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,785,772 (14.44 MB)

 Trainable params: 3,785,772 (14.44 MB)

 Non-trainable params: 0 (0.00 B)

In [75]:
model_wk3.fit(X, y,
              validation_data=(X_test, y_test),
              batch_size=1028,
              epochs=10,
              callbacks=[checkpoint, early_stop])

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/backend/torch/nn.py:466: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1036.)
  outputs = tnn.conv1d(


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 112ms/step - accuracy: 0.7337 - loss: 1.5487 - val_accuracy: 0.8680 - val_loss: 0.3818
Epoch 2/10
 2/20 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.8688 - loss: 0.3661

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with recall@precision available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_rec@prec` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss
  current = self.get_monitor_value(logs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.8896 - loss: 0.3194 - val_accuracy: 0.9257 - val_loss: 0.2537
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.9301 - loss: 0.2364 - val_accuracy: 0.9371 - val_loss: 0.2045
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step - accuracy: 0.9392 - loss: 0.1944 - val_accuracy: 0.9406 - val_loss: 0.1801
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.9412 - loss: 0.1730 - val_accuracy: 0.9419 - val_loss: 0.1639
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.9428 - loss: 0.1525 - val_accuracy: 0.9481 - val_loss: 0.1489
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.9558 - loss: 0.1300 - val_accuracy: 0.9584 - val_loss: 0.1298
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.9659 - loss: 0.1005 - val_accuracy: 0.9624 - val_loss: 0.1132
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - accuracy: 0.9734 - loss: 0.0721 - val_accuracy: 0.9679 - val_

In [76]:
# результаты модели wk3
pred_model_wk3 = model_wk3.predict(X_test).argmax(2)

print(classification_report(y_test.reshape(-1), pred_model_wk3.reshape(-1),
                            labels=list(id2labels.keys()),
                            target_names=list(id2labels.values()),
                            zero_division=0))

313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step
              precision    recall  f1-score   support

       B-LOC       0.63      0.44      0.52      4560
       B-ORG       0.37      0.45      0.41      4074
       B-PER       0.70      0.60      0.65      3542
       I-LOC       0.69      0.40      0.50      3060
       I-ORG       0.64      0.70      0.67      8008
       I-PER       0.88      0.70      0.78      7544
           O       0.87      0.93      0.90     40480
         PAD       1.00      1.00      1.00    468732

    accuracy                           0.97    540000
   macro avg       0.72      0.65      0.68    540000
weighted avg       0.97      0.97      0.97    540000



### Предсказания модели 3

In [77]:
label_seq('Лада припарковалась возле торгового центра.', word2id, id2labels,
          MAX_LEN, model_wk3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


[('Лада', 'I-ORG'),
 ('припарковалась', 'I-ORG'),
 ('возле', 'B-ORG'),
 ('торгового', 'I-ORG'),
 ('центра', 'I-ORG'),
 ('.', 'O')]

In [78]:
label_seq('Том Пушкина был поставлен на самое видное место', word2id, id2labels,
          MAX_LEN, model_wk3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step


[('Том', 'I-PER'),
 ('Пушкина', 'O'),
 ('был', 'O'),
 ('поставлен', 'O'),
 ('на', 'O'),
 ('самое', 'O'),
 ('видное', 'I-LOC'),
 ('место', 'O')]

In [80]:
label_seq('Вышка должна была столкнуться с рядом неприятностей.', word2id, id2labels,
          MAX_LEN, model_wk3)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step


[('Вышка', 'O'),
 ('должна', 'O'),
 ('была', 'O'),
 ('столкнуться', 'O'),
 ('с', 'O'),
 ('рядом', 'O'),
 ('неприятностей', 'O'),
 ('.', 'O')]

В целом и по метрикам, и по итоговым результатам model_wk3 лучше (она хотя бы что-то определила). Однако ряд слов, которые вообще ни по каким признаком не могут являться NE, определяются как NE (например, "припарковалась"). Удивило, что "Том" определилось как персона, хотя я была уверена, что модель справится с разграничением по смыслу.
В целом проблема либо в выбранных мною архитектурах (хотя со сверточными слоями обучение произошло лучше), либо в самом датасете (нужно бы его поизучать детальнее, чтобы понять, какие могут быть проблемы).